### Model Development Notebook

Model development would ideally be performed in the Snowflake DW. However, this notebook presents a quick alternative for that purpose.

In [1]:
import duckdb
import pandas as pd
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, Markdown

# Allow full column display
pd.set_option('display.max_columns', None)

# Helper to run DuckDB query
def run_query(sql_query: str) -> pd.DataFrame:
    db_path = Path('../data_lake/dbt.duckdb')
    try:
        with duckdb.connect(database=str(db_path), read_only=True) as con:
            return con.execute(sql_query).df()
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

print("Helper function `run_query` is defined.")

# Input and button
model_input = widgets.Text(
    value='dim_calendar',
    description='Model:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='300px', margin='0 10px 0 0')
)

run_button = widgets.Button(
    description='Load Model',
    button_style='primary',
    layout=widgets.Layout(width='120px', height='35px')
)

# Outputs
model_output = widgets.Output(layout=widgets.Layout(width='700px'))  # fixed width
tables_output = widgets.Output(layout=widgets.Layout(width='320px'))  # fixed width

# Preload table list
with tables_output:
    display(Markdown("### Available Tables/Models in `main` schema"))
    display(run_query("SHOW TABLES"))

# Button callback
def on_run_clicked(b):
    with model_output:
        model_output.clear_output()
        model_name = model_input.value

        display(Markdown(f"<h3 style='color:#5C2D91;'>Loading Model: <code style='font-size: 0.8em; background-color:#eee;'>{model_name}</code></h3>"))

        transactions_df = run_query(f"SELECT * FROM main.{model_name}")

        display(Markdown("#### Schema (Data Types)"))
        schema_df = pd.DataFrame({
            'Column': transactions_df.columns,
            'Non-Null Count': transactions_df.notnull().sum().values,
            'Dtype': transactions_df.dtypes.values
        })
        display(schema_df)

        display(Markdown("#### First 5 Rows"))
        display(transactions_df.head())

run_button.on_click(on_run_clicked)

# UI layout
input_row = widgets.HBox([model_input, run_button])

content_row = widgets.HBox(
    [model_output, tables_output],
    layout=widgets.Layout(
        justify_content='flex-start',
        align_items='flex-start',
        gap='10px',  # small gap
        width='100%'
    )
)

# Render full UI
display(widgets.VBox([input_row, content_row]))


Helper function `run_query` is defined.


In [2]:
sql_input = widgets.Textarea(
    value="""SELECT * FROM main.dim_calendar LIMIT 10""",
    placeholder='Enter your SQL query here...',
    description='SQL:',
    layout=widgets.Layout(width='1000px', height='250px'),
    style={'description_width': '40px'}
)

# Button to run SQL
run_sql_button = widgets.Button(
    description='Run SQL',
    button_style='success',
    layout=widgets.Layout(width='120px', height='40px')
)

# Output area
sql_output = widgets.Output()


# Remove jinja templates form dbt sql
def transform_dbt_sql(dbt_sql: str) -> str:
    """
    Transforms a dbt SQL query into a runnable DuckDB query by:
    1. Removing the dbt config block.
    2. Replacing {{ ref('model_name') }} with main.model_name.
    """
    # 1. Remove the config block using regex, handling multi-line configs
    sql_no_config = re.sub(r'\{\{.*?config\(.*?\)\s*\}\}', '', dbt_sql, flags=re.DOTALL | re.IGNORECASE)

    # 2. Replace ref() functions with the direct table name using regex
    final_sql = re.sub(r"\{\{\s*ref\s*\(\s*['\"](.*?)['\"]\s*\)\s*\}\}", r'main.\1', sql_no_config, flags=re.IGNORECASE)
    
    return final_sql.strip()

    
# Callback function
def on_sql_run_clicked(b):
    with sql_output:
        sql_output.clear_output()
        try:
            
            df = run_query(sql_input.value)
            display(Markdown("### Query Results"))
            display(df.head(10))  # Show top 10 rows
        except Exception as e:
            print(f"Error: {e}")

run_sql_button.on_click(on_sql_run_clicked)

# Display the full SQL UI
display(widgets.VBox([sql_input, run_sql_button, sql_output]))
